In [37]:
import h5py
import rasterio
from rasterio.mask import mask
from rasterio.transform import from_origin
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd 
import pandas as pd
from dnb_annual import *
from variables import years, composites, region_map, region_names

In [3]:
# this script is used only once to generate the regional images for each year
# country_polygons = gpd.read_file("geoBoundaries-UKR-ADM1.geojson")

# for year in years:
#     dnb = dnb_annual(year, composites, country_polygons)
#     dnb.load_all_data()
#     dnb.save_rasters()
#     dnb.load_rasters()
#     dnb.build_regional_images()
#     dnb.add_padding()
#     dnb.save_regional_images()

In [9]:
# this script is used to clean gdp data

# Inflation data
# inflation = pd.read_excel("data/isc_reg.xls", skiprows=2, header=1)
# inflation = inflation.drop(columns=inflation.columns[0])
# inflation = inflation.rename(columns={inflation.columns[-1]: "region"})
# inflation = inflation[~inflation["region"].isin(["Ukraine", "oblasts"])]
# inflation = inflation.dropna()
# inflation["region"] = inflation["region"].map(region_map)
# inflation.columns = inflation.columns.astype(str)
# inflation = inflation.melt(id_vars="region", var_name="year", value_name="inflation")
# inflation.to_csv("data/inflation.csv", index=False)

# GDP data
# gdp = pd.read_excel("data/ukr_reg_gdp.xls", skiprows=3, header=1)
# gdp = gdp.drop(columns=gdp.columns[0])
# gdp = gdp.iloc[:, np.r_[18:36, -1]]
# gdp = gdp.rename(columns={gdp.columns[-1]: "region"})
# gdp = gdp[~gdp["region"].isin(["Ukrane", "oblasts"])]
# gdp = gdp.dropna()
# gdp["region"] = gdp["region"].map(region_map)
# gdp["region"] = gdp["region"].fillna("Sevastopol")
# gdp.columns = gdp.columns.astype(str)
# gdp = gdp.rename(columns={gdp.columns[i]: gdp.columns[i][:4] for i in range(18)})
# gdp = gdp.melt(id_vars="region", var_name="year", value_name="real_gdp_change")

# # include only years from 2012 inclusive, exclude Sevastopol and the Autonomous Republic of Crimea
# gdp = gdp[gdp["year"].astype(int) >= 2012]
# gdp = gdp[~gdp["region"].isin(["Sevastopol", "Autonomous Republic of Crimea"])]

# # set the value for the starting year to 100 (2012), NaN for the rest
# gdp.loc[gdp["year"] == "2012", "real_gdp"] = 100
# gdp = gdp.sort_values(by=["region", "year"])
# gdp["real_gdp_change"] = gdp["real_gdp_change"] / 100

# # reste the index
# gdp = gdp.reset_index(drop=True)

# # # calculate the real gdp
# for i in range(1, gdp.shape[0]):

#     # skip if the year is 2012
#     if gdp.loc[gdp.index[i], "year"] == "2012":
#         continue
#     else:
#         gdp.loc[gdp.index[i], "real_gdp"] = gdp.loc[gdp.index[i-1], "real_gdp"] * (gdp.loc[gdp.index[i], "real_gdp_change"])

# # delete the real_gdp_change column
# gdp = gdp.drop(columns="real_gdp_change")

# # get the nominal gdp
# gdp_nominal = pd.read_excel("data/ukr_reg_gdp.xls", skiprows=3, header=1)
# gdp_nominal = gdp_nominal.iloc[:, np.r_[9, -1]]
# gdp_nominal.columns = ["gdp_nominal", "region"]
# gdp_nominal = gdp_nominal[~gdp_nominal["region"].isin(["Ukrane", "oblasts"])]
# gdp_nominal = gdp_nominal.dropna()
# gdp_nominal["region"] = gdp_nominal["region"].map(region_map)
# gdp_nominal["region"] = gdp_nominal["region"].fillna("Sevastopol")

# # merge nominal gdp to real gdp by region
# gdp = gdp.merge(gdp_nominal, on="region")

# # multiple the real gdp by the nominal gdp
# gdp["real_gdp"] = gdp["real_gdp"] * gdp["gdp_nominal"]

# # drop the nominal gdp column
# gdp = gdp.drop(columns="gdp_nominal")

# # for the region column, change all spaces to _
# gdp["region"] = gdp["region"].str.replace(" ", "_")

# # save the data
# gdp.to_csv("data/clean_ukr_gdp.csv", index=False)


In [10]:
# Prepare gdp data, Ukraine
gdp_ukr = pd.read_excel("data/ukr_reg_gdp.xls", skiprows=3, header=1)
gdp_ukr = gdp_ukr.drop(columns=gdp_ukr.columns[0])
gdp_ukr = gdp_ukr.iloc[:, np.r_[18:36, -1]]
gdp_ukr = gdp_ukr.rename(columns={gdp_ukr.columns[-1]: "region"})
gdp_ukr = gdp_ukr[~gdp_ukr["region"].isin(["Ukrane", "oblasts"])]
gdp_ukr = gdp_ukr.dropna()
gdp_ukr["region"] = gdp_ukr["region"].map(region_map)
gdp_ukr["region"] = gdp_ukr["region"].fillna("Sevastopol")
gdp_ukr.columns = gdp_ukr.columns.astype(str)
gdp_ukr = gdp_ukr.rename(columns={gdp_ukr.columns[i]: gdp_ukr.columns[i][:4] for i in range(18)})
gdp_ukr = gdp_ukr[~gdp_ukr["region"].isin(["Sevastopol", "Autonomous Republic of Crimea"])]

# select years from 2012 inclusive and the region column
gdp_ukr = gdp_ukr[[gdp_ukr.columns[i] for i in range(18) if int(gdp_ukr.columns[i]) >= 2012] + ["region"]]

# Poland
gdp_pol = pd.read_excel("data/pol_reg_gdp.xlsx", header=1, sheet_name=1)
gdp_pol = gdp_pol.drop(columns=gdp_pol.columns[0])
gdp_pol = gdp_pol.drop(0)
gdp_pol = gdp_pol.rename(columns={gdp_pol.columns[0]: "region"})
gdp_pol = gdp_pol.drop(columns="2022")

def clean_gdp_data(data):

    # set the 2012 first column to 100
    data["2012"] = 100

    # divide columns from 2013 to 2021 by 100
    for year in range(2013, 2022):
        data[str(year)] = data[str(year)] / 100

    # calculate the real gdp
    for i in range(2013, 2022):
        data[str(i)] = data[str(i)] * data[str(i-1)]

    # format to long
    data = data.melt(id_vars="region", var_name="year", value_name="real_gdp")
    data["region"] = data["region"].str.replace(" ", "_")
    
    return data

gdp_pol = clean_gdp_data(gdp_pol)
gdp_ukr = clean_gdp_data(gdp_ukr)

# save the data
gdp_pol.to_csv("data/clean_pol_gdp.csv", index=False)
gdp_ukr.to_csv("data/clean_ukr_gdp.csv", index=False)


### Neural Network

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout, BatchNormalization
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


# # Loading the MNIST dataset
# from keras.datasets import mnist
# (train_images, train_labels), (test_images, test_labels) = mnist.load_data()

c:\Users\jakub\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [15]:
# load clean gdp data
gdp = pd.read_csv("data/clean_ukr_gdp.csv")

# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(gdp), 765, 1076, 2))
y = np.zeros(len(gdp))

# load the snow covered and snow free images, add them together and append to the list
for i in range(len(gdp)):

    # get year, region, and gdp
    year = gdp["year"][i]
    region = gdp["region"][i]
    gdp_value = gdp["real_gdp"][i]

    # get the file name
    file_name = f"{year}_{region}.h5"

    # load the image
    file_path = f"data/annual_region_images/{file_name}"
    
    with h5py.File(file_path, 'r') as annual_region:
        # nearnad_snow_cov = annual_region["NearNadir_Composite_Snow_Covered"][:]
        # nearnad_snow_free = annual_region["NearNadir_Composite_Snow_Free"][:]
        # offnad_snow_cov = annual_region["OffNadir_Composite_Snow_Covered"][:]
        # offnad_snow_free = annual_region["OffNadir_Composite_Snow_Free"][:]
        allangle_snow_cov = annual_region["AllAngle_Composite_Snow_Covered"][:]
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]

        # add the two images together
        # combined = snow_covered + snow_free

    # add the gdp value to y
    y[i] = gdp_value

    # append both images as two channels to to X
    X[i, :, :, 0] = allangle_snow_cov
    X[i, :, :, 1] = allangle_snow_free
    # X[i, :, :, 2] = offnad_snow_cov
    # X[i, :, :, 3] = offnad_snow_free
    # X[i, :, :, 4] = nearnad_snow_cov
    # X[i, :, :, 5] = nearnad_snow_free
# print(X.shape)
# print(y.shape)

# Normalise the images
maximum = X.max()
X = X / maximum

# standardise gdp values
y_mean = y.mean()
y_std = y.std()
y = (y - y_mean) / y_std

# print(y.mean())
# print(maximum)

# change the format to a float16
X = X.astype(np.float16)
y = y.astype(np.float16)

In [16]:
# select 80% of the data for training, choose randomly
# X is the images, y is the gdp
train_size = int(0.8 * len(gdp))
test_size = len(gdp) - train_size

# select randomly train_size numbers from 0 to len(gdp)
train_indices = np.random.choice(len(gdp), train_size, replace=False)
test_indices = np.setdiff1d(np.arange(len(gdp)), train_indices)

# get the train data
X_train = X[train_indices]
y_train = y[train_indices]

# get the test data
X_test = X[test_indices]
y_test = y[test_indices]


In [17]:
model = Sequential()
# Resizing the images
model.add(Resizing(300, 440, input_shape=(765, 1076, 2)))
# Start with Convolutional layers
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))  # Additional Conv layer
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))  # Additional Conv layer
model.add(MaxPooling2D((2, 2)))
# Flatten the results to feed into a dense layer
model.add(Flatten())
# Add dense layers (hidden layers)
model.add(Dense(128, activation='relu'))  # Upscaled dense layer
model.add(Dense(64, activation='relu'))   # Additional dense layer
# Output layer
model.add(Dense(1))
# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

In [12]:
model = Sequential()
# Resizing the images
model.add(Resizing(300, 440, input_shape=(765, 1076, 2)))

# Start with Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

# Flatten the results to feed into a dense layer
model.add(Flatten())

# Add dense layers (hidden layers)
model.add(Dense(256, activation='relu'))  # Increased the number of neurons
model.add(Dense(128, activation='relu'))  # Increased the number of neurons
model.add(Dense(64, activation='relu'))   # Kept as is for detailed feature extraction

# Output layer
model.add(Dense(1))

# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

In [51]:
# check the size of X_train and y_train
print(X_train.shape)
print(X_train[1, :, :, 0].max())

(200, 765, 1076, 4)
0.0731


In [18]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)  # Assuming you have a validation split of 20%


Epoch 1/20


5/5 [==============================] - 12s 1s/step - loss: 1.2092 - mae: 0.6541 - val_loss: 0.6929 - val_mae: 0.4991
Epoch 2/20
5/5 [==============================] - 5s 1s/step - loss: 1.2174 - mae: 0.6176 - val_loss: 0.7056 - val_mae: 0.5114
Epoch 3/20
5/5 [==============================] - 6s 1s/step - loss: 1.2105 - mae: 0.6277 - val_loss: 0.7263 - val_mae: 0.5304
Epoch 4/20
5/5 [==============================] - 4s 904ms/step - loss: 1.1566 - mae: 0.6187 - val_loss: 0.9748 - val_mae: 0.7484
Epoch 5/20
5/5 [==============================] - 4s 875ms/step - loss: 1.1084 - mae: 0.6428 - val_loss: 0.7364 - val_mae: 0.5270
Epoch 6/20
5/5 [==============================] - 4s 826ms/step - loss: 0.9653 - mae: 0.5858 - val_loss: 0.6319 - val_mae: 0.5539
Epoch 7/20
5/5 [==============================] - 4s 844ms/step - loss: 0.6955 - mae: 0.5298 - val_loss: 0.5679 - val_mae: 0.5229
Epoch 8/20
5/5 [==============================] - 4s 807ms/step - loss: 0.7927 - mae: 0.5148 - v

In [19]:
# Step 6: Evaluate your model on the testing data
test_loss, test_mae = model.evaluate(X_test, y_test)
print('Test MAE:', test_mae) # mean absolute error
print('Test Loss:', test_loss)

2/2 [==============================] - 0s 185ms/step - loss: 0.2886 - mae: 0.3669
Test MAE: 0.36693692207336426
Test Loss: 0.2886132001876831


In [69]:
y_hat = model.predict(X_test).flatten()

# print the mean absolute percentage error
print(np.mean(100*np.abs((y_test - y_hat) / y_test)))

2/2 [==============================] - 3s 231ms/step
109.363976


In [9]:
# get the predictions from X_test
y_hat = model.predict(X_test).flatten()

# convert the predictions back to the original scale
# y_hat = y_hat * y_std + y_mean
# y_test = y_test * y_std + y_mean

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))
print(mae)

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))
print(percentage_error)


2/2 [==============================] - 0s 56ms/step
0.10076753842329474
36.946041487261475


## Recurrent NN

In [ ]:
# Preprocess the data

# load clean gdp data
gdp = pd.read_csv("data/clean_gdp.csv")

# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(gdp), 765, 1076, 4))
y = np.zeros(len(gdp))

## Random Forest and other

In [38]:
# load the tabular data
data = pd.read_csv("data/tabular_data_ukraine.csv")

# turn the region column into a categorical variable using one hot encoding
data = pd.get_dummies(data, columns=["region"])


In [39]:
data.columns

Index(['year', 'gdp', 'nearnad_snow_cov_1', 'nearnad_snow_cov_2',
       'nearnad_snow_cov_3', 'nearnad_snow_cov_4', 'nearnad_snow_cov_5',
       'nearnad_snow_cov_6', 'nearnad_snow_cov_7', 'nearnad_snow_cov_8',
       'nearnad_snow_cov_9', 'nearnad_snow_cov_10', 'nearnad_snow_free_1',
       'nearnad_snow_free_2', 'nearnad_snow_free_3', 'nearnad_snow_free_4',
       'nearnad_snow_free_5', 'nearnad_snow_free_6', 'nearnad_snow_free_7',
       'nearnad_snow_free_8', 'nearnad_snow_free_9', 'nearnad_snow_free_10',
       'offnad_snow_cov_1', 'offnad_snow_cov_2', 'offnad_snow_cov_3',
       'offnad_snow_cov_4', 'offnad_snow_cov_5', 'offnad_snow_cov_6',
       'offnad_snow_cov_7', 'offnad_snow_cov_8', 'offnad_snow_cov_9',
       'offnad_snow_cov_10', 'offnad_snow_free_1', 'offnad_snow_free_2',
       'offnad_snow_free_3', 'offnad_snow_free_4', 'offnad_snow_free_5',
       'offnad_snow_free_6', 'offnad_snow_free_7', 'offnad_snow_free_8',
       'offnad_snow_free_9', 'offnad_snow_free_10', 'al

In [42]:
# train a random forest model
from sklearn.ensemble import RandomForestRegressor

# drop year and gdp columns
X = data.drop(columns=["year", "gdp"])
y = data["gdp"]

# standardise gdp
# y_mean = y.mean()
# y_std = y.std()
# y = (y - y_mean) / y_std

# take randomly 80% of the data for training
train_size = int(0.8 * len(data))
test_size = len(data) - train_size

train_indices = np.random.choice(len(data), train_size, replace=False)
test_indices = np.setdiff1d(np.arange(len(data)), train_indices)

X_train = X.iloc[train_indices]
y_train = y.iloc[train_indices]
X_test = X.iloc[test_indices]
y_test = y.iloc[test_indices]

# train the model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train, y_train)

# get the predictions
y_hat = model.predict(X_test)

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

In [43]:
print(mae)
print(percentage_error)

5.596800568521989
8.205373678630673


In [45]:
# fit a lasso regression model 
from sklearn.linear_model import Lasso

model = Lasso(alpha=0)

model.fit(X_train, y_train)

# get the predictions
y_hat = model.predict(X_test)

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

print(mae)
print(percentage_error)


10.587519029167304
13.155784903890575


C:\Users\jakub\AppData\Local\Temp/ipykernel_25756/1922239168.py:6: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  model.fit(X_train, y_train)
c:\Users\jakub\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\Users\jakub\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5351.197506294976, tolerance: 6.679875229241091
  model = cd_fast.enet_coordinate_descent(


In [46]:
# fit a XGBoost model
import xgboost as xgb

model = xgb.XGBRegressor(objective="reg:squarederror", random_state=0)

model.fit(X_train, y_train)

# get the predictions
y_hat = model.predict(X_test)

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

print(mae)
print(percentage_error)

6.108772929030119
9.519860976604514


In [47]:
# fit a neural network model
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout, BatchNormalization
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# turn X_train bool to integers
boolean_columns = [col for col in X_train.columns if X_train[col].dtype == bool]

for col in boolean_columns:
    X_train[col] = X_train[col].astype(int) 
 

# fit the model on the X_train and y_train
model = Sequential()
model.add(Dense(256, activation="relu", input_dim=X_train.shape[1]))  # More complex first layer
model.add(Dropout(0.3))  # Dropout to reduce overfitting
model.add(Dense(128, activation="relu"))  # Second layer
model.add(Dropout(0.2))  # Additional Dropout layer
model.add(Dense(64, activation="relu"))  # Third layer
model.add(Dense(32, activation="relu"))  # Fourth layer
model.add(Dropout(0.1))  # Dropout layer
model.add(Dense(16, activation="relu"))  # New additional layer
model.add(Dense(1)) 

model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# get the predictions
boolean_columns = [col for col in X_test.columns if X_test[col].dtype == bool]
for col in boolean_columns:
    X_test[col] = X_test[col].astype(int)

y_hat = model.predict(X_test).flatten()

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

print(mae)
print(percentage_error)

C:\Users\jakub\AppData\Local\Temp/ipykernel_25756/3771872762.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].astype(int)
C:\Users\jakub\AppData\Local\Temp/ipykernel_25756/3771872762.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].astype(int)
C:\Users\jakub\AppData\Local\Temp/ipykernel_25756/3771872762.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

Epoch 1/100
5/5 [==============================] - 2s 61ms/step - loss: 9529.0703 - mae: 95.7070 - val_loss: 9911.1758 - val_mae: 98.6251
Epoch 2/100
5/5 [==============================] - 0s 15ms/step - loss: 9413.0283 - mae: 95.0958 - val_loss: 9724.5254 - val_mae: 97.6727
Epoch 3/100
5/5 [==============================] - 0s 14ms/step - loss: 9180.1504 - mae: 93.8590 - val_loss: 9358.8301 - val_mae: 95.7784
Epoch 4/100
5/5 [==============================] - 0s 14ms/step - loss: 8697.2891 - mae: 91.2676 - val_loss: 8624.9902 - val_mae: 91.8606
Epoch 5/100
5/5 [==============================] - 0s 15ms/step - loss: 7765.0288 - mae: 85.9637 - val_loss: 7238.7290 - val_mae: 83.9645
Epoch 6/100
5/5 [==============================] - 0s 14ms/step - loss: 6009.0005 - mae: 74.9440 - val_loss: 4871.8999 - val_mae: 68.5947
Epoch 7/100
5/5 [==============================] - 0s 12ms/step - loss: 3421.8848 - mae: 55.9977 - val_loss: 1709.7942 - val_mae: 40.4868
Epoch 8/100
5/5 [=================

C:\Users\jakub\AppData\Local\Temp/ipykernel_25756/3771872762.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col] = X_test[col].astype(int)
C:\Users\jakub\AppData\Local\Temp/ipykernel_25756/3771872762.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col] = X_test[col].astype(int)
C:\Users\jakub\AppData\Local\Temp/ipykernel_25756/3771872762.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 